In [12]:
!pip install ortools
!pip install dtale

     |████████████████████████████████| 7.6MB 1.5MB/s 
     |████████████████████████████████| 1.8MB 43.7MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
     |████████████████████████████████| 112kB 39.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 645kB 42.8MB/s 
     |████████████████████████████████| 358kB 44.3MB/s 
     |████████████████████████████████| 1.2MB 45.3MB/s 
     |████████████████████████████████| 3.4MB 42.6MB/s 
     |████████████████████████████████| 194kB 43.7MB/s 
     |████████████████████████████████| 1.8MB 44.2MB/s 
  Created wheel for dash-bootstrap-components: filename=dash_bootstrap_components-0.10.5-cp36-none-any.whl size=180066 sha256=23f9db71a4e09f1800b9fe5cf680ec136bd5812e4b04a3cc08b096f2388bcb68
  Stored in directory: /root/.cache/pip/wheels/41/7a/35/3c7ab615003bb9db

In [37]:
import pandas as pd
import numpy as np
import collections
#from ortools.sat.python import cp_model
#from dtale import show
#import dtale.app as dtale_app
#dtale_app.USE_COLAB = True
# Clone the entire repo.
!git clone -l -s https://github.com/mswastik/optimization.git cloned-repo
%cd cloned-repo

Cloning into 'cloned-repo'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 68 (delta 17), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (68/68), done.
/content/cloned-repo/cloned-repo


In [38]:
df=pd.read_excel('orders.xlsx')
ma=pd.read_excel('item master.xlsx')
bu=pd.read_excel('item master.xlsx',sheet_name='bulk')
pa=pd.read_excel('item master.xlsx',sheet_name='packing')

In [39]:
df = df[df['Line']=='Tmmthpkhti Limi']
df = df[df['Plant']==1024]
del df['Line']
df['due'] = df['Dispatch Date'] - df['SO Date'].min()
df['due'] = df['due'].dt.days
df['sodate'] = df['SO Date'] - df['SO Date'].min()
df['sodate'] = df['sodate'].dt.days
df.reset_index(inplace=True,drop=True)
df['Key']=df['SO'].astype('str')+df['FG Code']
del df['Dispatch Date']
del df['SO Date']
del df['Description']
del df['Customer']
df

,SO,FG Code,Order Qty,Plant,due,sodate,Key
0,17023287,gd29904300iX,486.0,1024,223,146,17023287gd29904300iX
1,17023287,gd29907800iX,216.0,1024,223,146,17023287gd29907800iX
2,17023287,gd39000800iX,360.0,1024,223,146,17023287gd39000800iX
3,17023287,gd55102400iX,180.0,1024,223,146,17023287gd55102400iX
4,17023572,gd43208500iX,756.0,1024,225,170,17023572gd43208500iX
...,...,...,...,...,...,...,...
179,17023468,gd28007800iX,336.0,1024,307,246,17023468gd28007800iX
180,17023468,gd28001700iX,640.0,1024,307,246,17023468gd28001700iX
181,17023833,gd55001700iX,320.0,1024,307,245,17023833gd55001700iX
182,17023833,gd55102400iX,360.0,1024,307,245,17023833gd55102400iX


In [32]:
df=df[df['FG Code']!='gC25010000kg']
df=df[df['FG Code']!='gC25020000kg']

In [40]:
jobs =df['Key']
op =[1,2,3,4]
mc = [1,2,3,4,5,6]
horizon = max(df["due"])
due = df.set_index('Key')

In [41]:
df2=df[['Key','FG Code','Order Qty']]
df2=df2.merge(pa[["FG Code","Line","Output"]], on="FG Code")
df2["duration"]=df2["Order Qty"]/df2["Output"]
del df2["Order Qty"]
df2.set_index(['Key','Line'],inplace=True)
del df2['FG Code']
del df2['Output']

In [42]:
df2['duration']=df2['duration'].fillna(0).astype('int')

In [13]:
from dtale import show
import dtale.app as dtale_app
dtale_app.USE_COLAB = True
#show(jobs)

https://n6ekujl3anh-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/1

In [15]:
show(pa)

https://n6ekujl3anh-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/3

In [15]:
due['due'].loc['17023833gd30212200iX']

307

In [10]:
from ortools.sat.python import cp_model
model = cp_model.CpModel()

In [52]:
sch = dict.fromkeys([1,2,3,4,5,6],[])
delay = []
for j in jobs:
    for m in mc:
        suffix = '_%s_%i' % (j,m)
        start = model.NewIntVar(0, horizon, "start"+suffix)
        end = model.NewIntVar(0, horizon, "end"+suffix)
        duration = model.NewIntervalVar(start, df2.loc[(j,m)].values[0], end, "duration"+suffix)
        delay.append(due['due'].loc[j]-end)
        sch[m].append(duration)

In [51]:
sch.update?

In [53]:
for m in mc:
        model.AddNoOverlap(sch[m])

In [54]:
model.Minimize(delay)

TypeError: ignored

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)